<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Definições-Iniciais" data-toc-modified-id="Definições-Iniciais-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Definições Iniciais</a></span></li><li><span><a href="#Configurando-E-mail" data-toc-modified-id="Configurando-E-mail-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configurando E-mail</a></span></li></ul></div>

Este notebook tem como objetivo realizar testes com a biblioteca `exchangelib` do Python para o envio de e-mails utilizando o Microsoft Exchange. A partir dela, pretende-se realizar configurações de envio de e-mails simples e, de forma incremental, analisar possibilidades de formatações adicionais nos objetos enviados.

# Definições Iniciais

In [172]:
# Importando bibliotecas
from dotenv import load_dotenv
import os
from exchangelib import Credentials, Account, Configuration, Message, DELEGATE, FileAttachment
import pandas as pd
import io

In [159]:
# Lendo variáveis de ambiente
load_dotenv()

# Definindo variáveis do projeto
USERNAME = 'thipanini94@gmail.com'
PWD = os.getenv('PWD')
SERVER = 'outlook.office365.com'
MAIL_BOX = 'thipanini94@gmail.com'
MAIL_TO = ['thipanini94@gmail.com']

# Variáveis testes de anexo de arquivos
CSV_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\01_german_credit_risk\german_credit_data.csv'
PNG_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\01_german_credit_risk\confusion_matrix_final.png'
TXT_FILEPATH = r'D:\Users\thiagoPanini\data-science-repos\kaggle\04_brazilian_e-commerce\dataset_test.txt'
XLSX_FILEPATH = r'C:\Users\thipa\Desktop\certificacao_alteryx\core\praticas\4701229_3BJP240W.xlsx'

# Configurando E-mail

In [35]:
# Definindo função de conexão com o servidor
def connect_exchange(username, password, server, mail_box, auto_discover=False, access_type=DELEGATE):
    """
    Realiza a conexão com o servidor através da conta exchange
    
    Parâmetros
    ----------
    :param username: e-mail do usuário com acesso de envio de e-mail do endereço smtp [type: string]
    :param password: senha do usuário com acesso de envio de e-mail do endereço smtp [type: string]
    :param server: servidor responsável por gerenciar o transporte [type: string]
    :param mail_box: endereço primário associado a conta de envio [type: string]
    :param auto_discover: flag para apontar ao EWS utilizando protocolo específico [type: bool, default=False]
    :param access_type: tipo de acesso relacionado as credenciais [type: obj, default=DELEGATE]
    
    Retorno
    -------
    :return account: objeto contendo informações compiladas de uma conta de usuário
    """
    
    # Definindo e configurando credenciais
    creds = Credentials(username=username, password=password)
    config = Configuration(server=server, credentials=creds)
    account = Account(primary_smtp_address=mail_box, credentials=creds,
                      autodiscover=auto_discover, access_type=access_type, config=config)
    
    return account

In [36]:
# Configurando conta de envio
account = connect_exchange(username=USERNAME, password=PWD, server=SERVER, mail_box=MAIL_BOX)

In [ ]:
# Criando função para envio de arquivos anexo
def attach_files(file_name, file_ext):
    
    # Lendo arquivo externo
    

OBS: Não será necessário considerar a leitura de arquivos externos, visto q isso pode ser feito diretamente no Alteryx através da leitura de qlqr base como insumo da macro.

In [158]:
# Lendo um arquivo qualquer
df = pd.read_csv(CSV_FILEPATH, encoding='utf-8')
df = df.iloc[:, 1:]

# Criando buffer para arquivo csv
buffer = io.BytesIO()
df.to_csv(buffer)

# Lendo buffer
attachments = []
buffer_content = buffer.getvalue()
attachments.append(('base_buffer.csv', buffer_content))

# Preparando mensagem
m = Message(account=account,
            subject='Teste DataFrame',
            body='Teste',
            to_recipients=MAIL_TO)

# Enviando anexo
for name, content in attachments or []:
    file = FileAttachment(name=name, content=content)
m.attach(file)

# Enviando mensagem
m.send_and_save()

In [164]:
df = pd.read_excel(XLSX_FILEPATH)
df

,emails,Entries
0,Odilia95@askjeeves.com,35
1,Conrad22@hotmail.com,35
2,Constantine37@askjeeves.com,35
3,Godwin75@hotmail.com,34
4,Wade18@yahoo.com,34
...,...,...
101,Rhoslyn89@gmail.com,1
102,Alistair88@hotmail.com,1
103,Milo95@hotmail.com,1
104,Emma96@live.com,0


In [168]:
import io

# Lendo um arquivo qualquer
df = pd.read_excel(XLSX_FILEPATH)

# Criando buffer para arquivo csv
buffer = io.BytesIO()
df.to_excel(buffer)

# Lendo buffer
attachments = []
buffer_content = buffer.getvalue()
attachments.append(('base_excel.xlsx', buffer_content))
buffer.close()

# Preparando mensagem
m = Message(account=account,
            subject='Teste DataFrame Excel',
            body='Teste',
            to_recipients=MAIL_TO)

# Enviando anexo
for name, content in attachments or []:
    file = FileAttachment(name=name, content=content)
m.attach(file)

# Enviando mensagem
m.send_and_save()

In [107]:
# Lendo um arquivo qualquer
df = pd.read_csv(CSV_FILEPATH, encoding='utf-8')
df = df.iloc[:, 1:]

# Salvando localmente para leitura de bytes
df.to_csv('attach_tmp.csv', index=False)
with open('attach.csv', 'rb') as f:
    content = f.read()
os.remove('attach.csv')

# Enviando anexo
attachments = []
attachments.append(('base.xlsx', content))